# MODFLOW-NWT Agricultural Water Use package (AG) example

The Agricultural water use package allows users to specify irrigation from wells and stream segments within MODFLOW-NWT and GSFLOW.

In this notebook examples of how to read in the AG package using pygsflow are presented

In [1]:
import os
import gsflow
import flopy
import numpy as np
import matplotlib.pyplot as plt

## Loading a pre-built AG package file

We can load the AG package as part of a GSFLOW or modflow model automatically, or load the AG package individually using the `ModflowAg` class.

Here we present the second option for illustration.

In [2]:
# define workspace and our ag_file name
workspace = "./data/sagehen/modflow"
ag_file = "sagehen.awu"
dis_file = "saghen_ag.dis"

Let's create an empty `Modflow` model object so we can begin adding packages to it

In [3]:
ml = gsflow.modflow.Modflow(modelname="sagehen_example", model_ws=workspace)
ml

MODFLOW 0 layer(s), 0 row(s), 0 column(s), 0 stress period(s)

Next we can add the DIS file to the `Modflow` object. This will give us information about the model size, and the length of the model. 

### Note: We are going to use `flopy` to read in the dis file and add it to the `Modflow` object

In [4]:
dis = flopy.modflow.ModflowDis.load(os.path.join(workspace, dis_file), model=ml)
ml

MODFLOW 2 layer(s), 77 row(s), 84 column(s), 344 stress period(s)

From the output, we can see that the `ModflowDis` object has been loaded and added to the `Modflow` model

### Now let's load the AG package using pygsflow's `ModflowAg.load()` method

The `load()` method has a few important parameters.

`f` : the AG package file name

`model` : the `gsflow.modflow.Modflow` model object

`nper` : number of stress periods in the model

`method` : can be `gsflow` or `modflow`, if this is being used as part of a `gsflow` project, select `gsflow` (default)


In [5]:
ag = gsflow.modflow.ModflowAg.load(os.path.join(workspace, ag_file), model=ml, nper=dis.nper, method='gsflow')
# now let's look at the well_list
ag.well_list

rec.array([(1, 38, 53, -500.), (1, 37, 50, -500.), (1, 37, 51, -500.),
           (1, 37, 52, -500.), (1, 37, 54, -500.), (1, 37, 55, -500.),
           (1, 37, 56, -500.), (1, 37, 57, -500.), (1, 37, 58, -500.),
           (1, 36, 50, -500.), (1, 36, 51, -500.), (1, 36, 52, -500.),
           (1, 36, 55, -500.), (1, 35, 50, -500.), (1, 35, 56, -500.),
           (1, 35, 57, -500.), (1, 35, 58, -500.), (1, 34, 52, -500.),
           (1, 34, 53, -500.), (1, 34, 55, -500.), (1, 34, 56, -500.),
           (1, 34, 57, -500.), (1, 34, 58, -500.), (1, 33, 48, -500.),
           (1, 33, 49, -500.), (1, 33, 50, -500.), (1, 33, 51, -500.),
           (1, 33, 52, -500.), (1, 33, 54, -500.), (1, 33, 55, -500.),
           (1, 33, 56, -500.), (1, 33, 57, -500.), (1, 33, 58, -500.),
           (1, 33, 53, -500.)],
          dtype=[('k', '<i4'), ('i', '<i4'), ('j', '<i4'), ('flux', '<f8')])

### the well_list is stored as a flopy compatible recarray, we can also look at some of the other datatypes

other datatypes include:

`irrdiversion` which hold the irrdiversion blocks from the AG package

`irrwell` which holds the irrwell blocks from the AG package

`supwell` which holds the supwell blocks from the AG package

In [6]:
# irrdiversion
irs = ag.irrdiversion

# irrwell
irr = ag.irrwell

# supwell
sup = ag.supwell

print(irr)

{0: rec.array([],
          dtype=[('wellid', '<i4'), ('numcell', '<i4'), ('period', '<f8'), ('triggerfact', '<f8'), ('hru_id0', '<i4'), ('dum0', '<i4'), ('eff_fact0', '<f8'), ('field_fact0', '<f8')]), 1: array([],
      dtype=(numpy.record, [('wellid', '<i4'), ('numcell', '<i4'), ('period', '<f8'), ('triggerfact', '<f8'), ('hru_id0', '<i4'), ('dum0', '<i4'), ('eff_fact0', '<f8'), ('field_fact0', '<f8')])), 2: array([],
      dtype=(numpy.record, [('wellid', '<i4'), ('numcell', '<i4'), ('period', '<f8'), ('triggerfact', '<f8'), ('hru_id0', '<i4'), ('dum0', '<i4'), ('eff_fact0', '<f8'), ('field_fact0', '<f8')])), 3: array([],
      dtype=(numpy.record, [('wellid', '<i4'), ('numcell', '<i4'), ('period', '<f8'), ('triggerfact', '<f8'), ('hru_id0', '<i4'), ('dum0', '<i4'), ('eff_fact0', '<f8'), ('field_fact0', '<f8')])), 4: array([],
      dtype=(numpy.record, [('wellid', '<i4'), ('numcell', '<i4'), ('period', '<f8'), ('triggerfact', '<f8'), ('hru_id0', '<i4'), ('dum0', '<i4'), ('eff_fact0

## Now let's illustrate how to build a AG package from scratch

First let's get some empty data-types to store data in using `ModflowAG.get_empty()`

In [7]:
well_list = gsflow.modflow.ModflowAg.get_empty(numrecords=2, block="well")

Let's create 2 wells now and add it to our well_list 

In [8]:
# layer, row, column, maximum_flux
temp = [[1, 38, 53, -500.],
        [1, 38, 54, -1000.]]

for ix, well in enumerate(temp):
    well_list[ix] = tuple(well)
    
well_list

rec.array([(1, 38, 53,  -500.), (1, 38, 54, -1000.)],
          dtype=[('k', '<i4'), ('i', '<i4'), ('j', '<i4'), ('flux', '<f8')])

Now let's create two irrigation cells that are linked to our wells

In [9]:
irrwell0 = gsflow.modflow.ModflowAg.get_empty(numrecords=2, maxells=1, block="irrwell_gsflow")

# let's look at the dtypes we need 
irrwell0.dtype.names

('wellid',
 'numcell',
 'period',
 'triggerfact',
 'hru_id0',
 'dum0',
 'eff_fact0',
 'field_fact0')

Let's create two irrigation cell links and add it to our irrwell

Note: -1e+10 and 1e+10 are used to indicate that there is no data for this (unused variables), see Agricultural Water Use Package for MODFLOW-NWT for more infromation about the variables... 

In [10]:
# 'wellid', 'numcell', 'period', 'triggerfact', 'hru_id0', 'dum0', 'eff_fact0', 'field_fact0'
temp = [[1, 1, -1e+10, 1e+10, 2741, -1, 1., 0.5],
        [2, 1, -1e+10, 1e+10, 2743, -1, 1., 0.5]]

for ix, irr in enumerate(temp):
    irrwell0[ix] = tuple(irr)

irrwell0

rec.array([(1, 1, -1.e+10, 1.e+10, 2741, -1, 1., 0.5),
           (2, 1, -1.e+10, 1.e+10, 2743, -1, 1., 0.5)],
          dtype=[('wellid', '<i4'), ('numcell', '<i4'), ('period', '<f8'), ('triggerfact', '<f8'), ('hru_id0', '<i4'), ('dum0', '<i4'), ('eff_fact0', '<f8'), ('field_fact0', '<f8')])

Since irrwell is stress period data, this needs to be made for each stress period in the model and placed in a dictionary with the format `{kper : data}`

We are just going to reuse the same dataset for a three stress period example

In [11]:
kper = 3

irrwell = {}
for per in range(kper):
    irrwell[per] = irrwell0
    
irrwell

{0: rec.array([(1, 1, -1.e+10, 1.e+10, 2741, -1, 1., 0.5),
            (2, 1, -1.e+10, 1.e+10, 2743, -1, 1., 0.5)],
           dtype=[('wellid', '<i4'), ('numcell', '<i4'), ('period', '<f8'), ('triggerfact', '<f8'), ('hru_id0', '<i4'), ('dum0', '<i4'), ('eff_fact0', '<f8'), ('field_fact0', '<f8')]),
 1: rec.array([(1, 1, -1.e+10, 1.e+10, 2741, -1, 1., 0.5),
            (2, 1, -1.e+10, 1.e+10, 2743, -1, 1., 0.5)],
           dtype=[('wellid', '<i4'), ('numcell', '<i4'), ('period', '<f8'), ('triggerfact', '<f8'), ('hru_id0', '<i4'), ('dum0', '<i4'), ('eff_fact0', '<f8'), ('field_fact0', '<f8')]),
 2: rec.array([(1, 1, -1.e+10, 1.e+10, 2741, -1, 1., 0.5),
            (2, 1, -1.e+10, 1.e+10, 2743, -1, 1., 0.5)],
           dtype=[('wellid', '<i4'), ('numcell', '<i4'), ('period', '<f8'), ('triggerfact', '<f8'), ('hru_id0', '<i4'), ('dum0', '<i4'), ('eff_fact0', '<f8'), ('field_fact0', '<f8')])}

# Now let's start building our new `ModflowAg` object using the data we just created

### We will need to start by specifying the option block using the new `OptionBlock` utility in FloPy

let's create an empty options block and start filling out the data

In [12]:
options = flopy.utils.OptionBlock(options_line="", package=gsflow.modflow.ModflowAg)
options

OPTIONS
END

First let's specify the number of wells in the AG package (see Agricultural Water Use Package doc, for variable names)

In [13]:
options.maxwell = True
options.nummaxwell = 2

# we can see that the maxwell option and nummaxwell have been added to the options block
print(options)

OPTIONS
MAXWELL 2
END



Now let's specify our irriagation well information

In [14]:
options.irrigation_well = True
options.numirrwells = 2
options.maxcellswell = 1

options

OPTIONS
IRRIGATION_WELL 2 1
MAXWELL 2
END

Finally let's specify the unit number to write CBC output for wells to 

In [15]:
options.wellcbc = True
options.unitcbc = 99

options

OPTIONS
IRRIGATION_WELL 2 1
MAXWELL 2
WELLCBC 99
END

### Now we're ready to create our new `ModflowAg` package object

In [16]:
new_ag = gsflow.modflow.ModflowAg(ml, options=options, well_list=well_list, irrwell=irrwell)
new_ag

****Warning -- two packages of the same type:  <class 'gsflow.modflow.mfawu.ModflowAg'> <class 'gsflow.modflow.mfawu.ModflowAg'>
replacing existing Package...



    The ModflowAg class is used to build read, write, and edit data
    from the MODFLOW-NWT AG package.

    Parameters
    ----------
    model : gsflow.modflow.Modflow object
        model object
    extension : str, optional
        default is .ag
    options : flopy.utils.OptionBlock object
        option block utility from flopy
    unitnumber : list, optional
        fortran unit number for modflow, default 69
    filenames : list, optional
        file name for ModflowAg package to write input

     _name = AG
 _nper = 344 ('int)
 _parent = MODFLOW 2 layer(s), 77 row(s), 84 column(s), 344 stress period(s) ('gsflow.modflow.mf.Modflow)
 accel = None ('NoneType)
 acceptable_dtypes (list, items = 3
 allowDuplicates = False ('bool)
 bc_color = black ('str)
 diversionirrlist = False ('bool)
 diversionlist = False ('bool)
 etdemand = False ('bool)
 extra = 
 file_name = sagehen_example.ag
 fn_path = ./data/sagehen/modflow\sagehen_example.ag ('str)
 irrdiversion = None ('NoneType)
 ir

### Finally we can write the new_file to disk

In [17]:
new_ag.write_file()